<a href="https://colab.research.google.com/github/Radioactive009/Alumni-Link/blob/main/NLP_Lab_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
import string
import math
from collections import Counter, defaultdict

nltk.download('punkt')
nltk.download('brown')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [2]:
from nltk.corpus import brown

sentences = brown.sents()


Step 1.2 - Preprocess Text

In [3]:
def preprocess(sentences):
    tokens = []

    for sentence in sentences:
        for word in sentence:
            word = word.lower()

            # Remove punctuation
            if word not in string.punctuation:
                tokens.append(word)

    return tokens

tokens = preprocess(sentences)

print("Total words after preprocessing:", len(tokens))


Total words after preprocessing: 1034378


Task 2(a) -  Count Word Frequencies

In [4]:
word_counts = Counter(tokens)

print("Top 10 words:")
print(word_counts.most_common(10))


Top 10 words:
[('the', 69971), ('of', 36412), ('and', 28853), ('to', 26158), ('a', 23195), ('in', 21337), ('that', 10594), ('is', 10109), ('was', 9815), ('he', 9548)]


2(b) - Calculate unigram probability
P(w) = Count(w)/Total Words

In [5]:
total_words = len(tokens)

unigram_prob = {}

for word in word_counts:
    unigram_prob[word] = word_counts[word] / total_words


2(c) - Compute Sentence Probability

In [7]:
sentence = "the government is working"
def sentence_probability_unigram(sentence):
    words = sentence.lower().split()
    prob = 1

    for word in words:
        if word in unigram_prob:
            prob *= unigram_prob[word]
        else:
            prob *= 1e-6  # small value for unknown word

    return prob

print("Unigram Sentence Probability:",
      sentence_probability_unigram(sentence))


Unigram Sentence Probability: 3.8999802363629654e-11


Task 3 - Bigram Model

3(a) - Generate Bigram Pairs

In [8]:
def generate_bigrams(tokens):
    bigrams = []
    for i in range(len(tokens)-1):
        bigrams.append((tokens[i], tokens[i+1]))
    return bigrams

bigrams = generate_bigrams(tokens)


In [9]:
#Counting Bigram frequencies
def generate_bigrams(tokens):
    bigrams = []
    for i in range(len(tokens)-1):
        bigrams.append((tokens[i], tokens[i+1]))
    return bigrams

bigrams = generate_bigrams(tokens)


3b. Compute Bigram Probabilities
P(wi​∣wi−1​) = {Count(Wi-1, Wi)/Count(Wi-1)}

In [11]:
bigram_prob = {}

bigram_counts = Counter(bigrams)
for (w1, w2) in bigram_counts:
    bigram_prob[(w1, w2)] = bigram_counts[(w1, w2)] / word_counts[w1]

In [12]:
#Compute Sentence Probability (Bigram)
def sentence_probability_bigram(sentence):
    words = sentence.lower().split()
    prob = 1

    for i in range(1, len(words)):
        if (words[i-1], words[i]) in bigram_prob:
            prob *= bigram_prob[(words[i-1], words[i])]
        else:
            prob *= 1e-6

    return prob

print("Bigram Sentence Probability:",
      sentence_probability_bigram(sentence))


Bigram Sentence Probability: 3.8989836026935755e-08


Task 4 - Laplace Smoothing

In [13]:
vocab_size = len(word_counts)

def sentence_probability_bigram_laplace(sentence):
    words = sentence.lower().split()
    prob = 1

    for i in range(1, len(words)):
        count_bigram = bigram_counts[(words[i-1], words[i])]
        count_unigram = word_counts[words[i-1]]

        prob *= (count_bigram + 1) / (count_unigram + vocab_size)

    return prob

print("Bigram with Laplace:",
      sentence_probability_bigram_laplace(sentence))


Bigram with Laplace: 3.9563483935955733e-11


Task 5 - Perplexity

Step 5.1 - Split Train, Test data

In [14]:
train_sentences = sentences[:50000]
test_sentences = sentences[50000:51000]

train_tokens = preprocess(train_sentences)


Step 5.2 - Compute Perplexity

In [15]:
def perplexity(test_sentences):
    log_prob = 0
    N = 0

    for sentence in test_sentences:
        words = [w.lower() for w in sentence if w not in string.punctuation]

        for i in range(1, len(words)):
            count_bigram = bigram_counts[(words[i-1], words[i])]
            count_unigram = word_counts[words[i-1]]

            prob = (count_bigram + 1) / (count_unigram + vocab_size)

            log_prob += math.log(prob)
            N += 1

    return math.exp(-log_prob / N)

print("Perplexity:", perplexity(test_sentences))


Perplexity: 4752.49153940994
